Introduction.

Hey Kaggles! 
First competiton was a great fun and also a litte bit frustrating. I tried many techniques to prepare the dataset and I tried many algorithms to find the one. I also gridsearchCV for the best hyperparameters. Finally I used ensemble model to get even better results, but I end up only on top_55% in kaggle scoreboard. The top places in the first competition were decided by 4th and 5th place in decimal point. So I was wondering what else can be done to improve the final score. In this notebook I will try to learn and use one of the win-win competition algorithm Lightgbm according to  and see what happen.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# EDA
import pandas as pd
import numpy as np
from scipy.stats import skew

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import missingno

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# Evaluation
from sklearn.metrics import mean_squared_error

# Algorithms
import lightgbm as lgbm
from sklearn.linear_model import ElasticNetCV
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR


In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/test.csv")

In [ ]:
train

As we can see, this time we get a dataset with categorical and numerical type, therefore we have to deal with them separately.

In [ ]:
cat_feat = [x for x in train.columns if train[x].name.startswith("cat")]
num_feat = [x for x in train.columns if train[x].name.startswith("cont")]

## Feature engineering

### Numerical features

In [ ]:
skewed_features = train[num_feat].skew().sort_values(ascending=False)
skewed_index = skewed_features.index

In [ ]:
plt.figure(figsize=(8,5), dpi=110)
skewed_features.plot(kind="bar")
plt.title("Numerical features skewness");

Acceptable value of skewness is between -0.5 < x < 0.5. One of the method to remove outliers is by calculating their upper and lower limits using quantiles.

In [ ]:
fig = plt.figure(figsize=(18, 30))

for i, col in enumerate(num_feat):
    plt.subplot(10,3, i+1)
    sns.histplot(x=train[col], bins=20, kde=True, color='green')
    plt.tight_layout()
fig.show()

The distribution of continues features look-a-like bimodial or multimodial.

In [ ]:
def skewness(df):
    for col in num_feat:
        q3 = np.quantile(df[col], 0.75)
        q1 = np.quantile(df[col], 0.25)
        iqr = q3 - q1
        upper_limit = q3 + (1.5*iqr)
        lower_limit = q1 - (1.5*iqr)
        df = df[df[col].apply(lambda x: lower_limit <= x <= upper_limit)]
    return df

In [ ]:
train = skewness(train)

In [ ]:
# Target column
sns.displot(x=train['target'], kde=True)

In [ ]:
train['target'].describe()

In [ ]:
sns.displot(x=np.log1p(train['target']))

The distribution of our numeric features including target column is in most cases multimodial or bimodial. Now the question is what I should do about it? **need more reading**

In [ ]:
num_feat.append("target")

In [ ]:
corr = train[num_feat].corr()
fig = plt.figure(figsize=(12,8))
sns.heatmap(corr, annot=True, cmap=plt.cm.Reds)

There is some correlation between features but non of them have correletion with the target and to be frank in some cases the corelation is very weak. I am not sure if that is important factor which need to be somehow adress.

In [ ]:
# This plot might take some time to plot it
# sns.pairplot(data=train[num_feat])

### Categorical features

In [ ]:
# Different in the categorical values
for col in cat_feat:
    diff = set(train[col].unique()) - set(test[col].unique())
    #print(f"Train column {col} has unique: {train[col].unique()}")
    #print(f"Test column {col} has unique: {test[col].unique()}")
    print(f"Train and Test dataset :{col}  unique different is: {diff}")

As we can see there is a unique values difference between Train and Test dataset in column "cat6". We need to udress it before we onehotencode them. 

In [ ]:
fig = plt.figure(figsize=(18,20))

for i, col in enumerate(cat_feat):
    plt.subplot(8,3, i+1)
    sns.boxplot(x=col, y='target', data=train)
    plt.ylabel("target")
    plt.tight_layout()

After removing outliers from numerical features we also removed some outliers from categorical features, looks better now. However there is not to much variance between categorical features.

In [ ]:
fig = plt.figure(figsize=(18, 30))

for i, col in enumerate(cat_feat):
    plt.subplot(8,3, i+1)
    sns.countplot(x=train[col], palette="viridis_r")
    plt.tight_layout()
fig.show()

Note from this plot that most of the categorical features are unbalanced except feature cat1.

## From here we can jump to new approach section or continue with the first one

In [ ]:
train.describe(include='object').T

In [ ]:
all_data = pd.concat([train, test], axis=0)

missingno.matrix(all_data, figsize=(16,3))

In [ ]:
#drop_list = ["cat0", "cat2", "cat4", "cat6", "cat7"]
#all_data.drop(drop_list, axis=1, inplace=True)
#for cat in drop_list:
    #cat_feat.remove(cat)

So after removing unbalanced features our model performed worse, so that didn't help.

In [ ]:
# Dummy variables
dummy_data = pd.get_dummies(all_data[cat_feat], drop_first=True)

for col in cat_feat:
    all_data.drop(col, axis=1, inplace=True)
    
concat_data = pd.concat([dummy_data, all_data], axis=1)

In [ ]:
new_train = concat_data[concat_data['target'] >= 0]
new_test = concat_data[concat_data['target'].isna()]

In [ ]:
#for col in cat_feat:
    #train[col] = pd.Series(train[col], dtype="category")
    #test[col] = pd.Series(test[col], dtype="category")

In [ ]:
X = new_train.drop(["id", "target"], axis=1)
y = new_train['target']

I didn't scale numeric data as they seems to be in the same range and from the previous competition there was no model improvement. So for now I won't do it but I might to come back and change that here. 

In [ ]:
# Let's find the base model_performance

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
rmse_score = []


 
params ={"objective": "regression",
         "metric": "rmse",
         "verbosity": -1,
         "boosting_type": "gbdt",
         "feature_fraction": 0.5,
         "max_depth": 10,
         "num_leaves": 60,
         "lambda_l1": 2,
         "lambda_l2": 2,
         "learning_rate": 0.01,
         "min_child_samples":50,
         "bagging_fraction": 0.7,
         "bagging_freq": 1, 
         "max_bin": 80,}
          #"is_unbalance":True,
          #"subsample":0.3}
    
    
lgb_train = lgbm.Dataset(X_train, y_train)
lgb_val = lgbm.Dataset(X_val, y_val)
gbm = lgbm.train(params,
                 lgb_train,
                 valid_sets=[lgb_train, lgb_val],
                 num_boost_round=10000,
                 verbose_eval=100,
                 early_stopping_rounds=100,
                 )
    
# Extra Boosting
lgb_train = lgbm.Dataset(X_train, y_train)
lgb_val = lgbm.Dataset(X_val, y_val)
params = {"objective": "regression",
          "metrics": "rmse",
          "verbosity": -1,
          "boosting_type": "gbdt",
          "feature_fraction": 0.5,
          "max_depth": 10,
          "num_leaves":200,
          "lambda_l1": 2,
          "labmda_l2": 2,
          "learning_rate": 0.003,
          "min_child_samples": 50,
          "max_bin": 80,
          #"is_unbalance":True,
          #"subsample":0.3
          "bagging_fraction": 0.7,
          "bagging_freq": 1,}
    
gbm = lgbm.train(params,
                 lgb_train,
                 valid_sets = [lgb_train, lgb_val],
                 verbose_eval = 100,
                 num_boost_round = 10000,
                 early_stopping_rounds=100,
                 init_model = gbm)
    
y_pred = gbm.predict(X_val)
rmse_score.append(np.sqrt(mean_squared_error(y_val, y_pred)))

In [ ]:
id_col= new_test['id']
new_test = new_test.drop(["id","target"], axis=1)

In [ ]:
# Submmit to kaggle and see where we are.

In [ ]:
gbm1 = gbm.predict(new_test)

In [ ]:
first_sub = pd.DataFrame({"id":id_col,
                          "target": gbm1})

first_sub.to_csv("sub2_tbs_feb-2021.csv", index=False)

### GridSearchCV and KFolds for lightgbm

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [ ]:
# I've used these parameters first
#param_grid = {"reg_alpha":[0.1, 0.5, 1],
              #"max_depth": [5,8,10,20],
              #"num_leaves": [30,60,120],
              #"max_bin":[20, 40, 80],
              #"n_estimators":[100, 500, 1000]}

In [ ]:
#kf = KFold(n_splits=3, shuffle=True, random_state=42)

#param_grid = {"max_depth": [20,40],
              #"num_leaves": [120, 150, 200],
              #"max_bin":[80, 120],
              #"n_estimators":[1000, 1500]}

#lgbm_model = lgbm.LGBMRegressor(objective="regression",
                                #metrics='rmse',
                                #boosting_type='gbdt',
                                #feature_fraction=0.5,
                                #bagging_fraction=0.7,
                                #learning_rate=0.01,
                                #bagging_freq=1)

#grid_lgbm = GridSearchCV(lgbm_model,
                         #param_grid,
                         #scoring='neg_root_mean_squared_error'
                         #cv=kf,
                         #verbose=2,
                         #n_jobs=-1)

#grid_lgbm.fit(X_train, y_train)

In [ ]:
#grid_lgbm.best_score_

In [ ]:
#grid_lgbm.best_params_

These hyperparameters has been found by gridsearchCV:

{'max_bin': 120, 'max_depth': 40, 'n_estimators': 1500, 'num_leaves': 120}

My original strategy was to GridSearchCV best hyperparameters for chosen algorithms and than use stacking algorithm to lower my rmse. It is a good technique for a small datasets but with a dataset like this (300000,26) some algorithms e.g SVR or RandomForestRegressor takes forever and then on some point we running out of memory. So I think that we have 2 options. First, we could take a sample and than we could gridsearchCV. But here question arise if the sample we take is representative for a hole dataset. Second, we could find the right reguralization parameter for a given algorithm, that would reduce our computation time. Last, we could change the aproach completely, leave for now parameter tuning and start from the begining which is feature engineering. I just come across an article about discretization of continues features and I am dying to try it out and what effect that could be on performance as well as on computation time. 

### GridSearchCV and KFolds for LinearSVR

In [ ]:
from sklearn.svm import LinearSVR

Because of big O which is in this case (n_features * n_observations**2) for SVR algorithm I need to use LinearSVR as it is much faster for a big datasets.

In [ ]:
%%time
kf = KFold(n_splits=3, shuffle=True, random_state=42)

param_grid = {"loss": ["epsilon_insensitive", "squared_epsilon_insensitive"],
              "C": [1,5, 10, 100,]}

lsvr_model = LinearSVR()

grid_svr = GridSearchCV(lsvr_model,
                        param_grid, 
                        verbose=2,
                        scoring='neg_root_mean_squared_error',
                        cv=kf,
                        n_jobs=-1)
grid_svr.fit(X_train, y_train)

In [ ]:
grid_svr.best_params_

{'C': 1, 'loss': 'squared_epsilon_insensitive'}

In [ ]:
params = grid_svr.best_params_

In [ ]:
l_svr = LinearSVR(**params)
l_svr.fit(X_train, y_train)

In [ ]:
y_pred_lrsvr = l_svr.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred_lrsvr))
rmse

### RandomizedSearchCV for XGBoost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)

param_grid = {"objective":["reg:squarederror"],
              "max_depth": [6, 10, 14],
              "n_estimators": [500, 1000, 1500]}

xgboost_model = XGBRegressor()

random_xgboost = RandomizedSearchCV(xgboost_model,
                                    param_grid,
                                    scoring="neg_root_mean_squared_error"
                                    verbose=2,
                                    cv = kf,
                                    n_jobs=-1)


random_xgboost.fit(X_train,y_train)

In [ ]:
xgboost_model = XGBRegressor(objective="reg:squarederror")

xgboost_model.fit(X_train, y_train)

In [ ]:
y_pred_xgb = xgboost_model.predict(X_val)

rmse = np.sqrt(mean_squared_error(y_val, y_pred_xgb))
rmse

### RandomForestRegressor with default parameters

I tried to GridSearchCV for only these very limited hyperparameters and I couldn't complete search.

param_grid = {"n_estimators":[50,100,500],
              "max_features": ["auto", "sqrt", "log2"],
              "bootstrap": [True, False]}


In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfr_model = RandomForestRegressor()

rfr_model.fit(X_train, y_train)

In [ ]:
y_pred_rfr = rfr_model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred_rfr))
print(f"Random Forest Regressor")
print(f"RMSE: {rmse}")

### ElastiNetCV

In [ ]:
from sklearn.linear_model import ElasticNetCV

In [ ]:
el_model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1])
el_model.fit(X_train, y_train)

y_pred_el = el_model.predict(X_val)
rmse_el = np.sqrt(mean_squared_error(y_val, y_pred_el))
print(f"ElasticNetCV")
print(f"RMSE: {rmse_el}")

In [ ]:
el_model.l1_ratio_

### Stacking model

In [ ]:
from sklearn.ensemble import StackingRegressor

## Stacking Regressor

In [ ]:
# Define the base models
level_0 = list()
level_0.append(("el_model", ElasticNetCV(l1_ratio=1.0)))
level_0.append(("rfr_model", RandomForestRegressor()))
level_0.append(("xgboost", XGBRegressor(objective="reg:squarederror")))
level_0.append(("linearSVR", LinearSVR(**params)))

# Define meta learner model
level_1 = lgbm.LGBMRegressor(objective="regression",
                             metrics='rmse',
                             boosting_type='gbdt',
                             feature_fraction=0.5,
                             bagging_fraction=0.7,
                             learning_rate=0.01,
                             bagging_freq=1, 
                             max_bin=120,
                             max_depth=40,
                             n_estimators=1500,
                             num_leaves=120)

# Instantiate ensemble model
ens_model = StackingRegressor(estimators=level_0, 
                              final_estimator=level_1,
                              cv=kf,
                              verbose=2)

# Fit the model
ens_model.fit(X_train, y_train)

In [ ]:
y_pred_ens = ens_model.predict(X_val)
rmse_ens = np.sqrt(mean_squared_error(y_val, y_pred_ens))
print(f"Stacking Regressor")
print(f"RMSE: {rmse_ens}")

#Submmit to Kaggle

In [ ]:
ens_model.fit(X,y)


ens_pred = ens_model.predict(new_test)

In [ ]:
stack_sub = pd.DataFrame({"id": id_col,
                         "target": ens_pred})

stack_sub.to_csv("enemble_model_sub_tbs.csv", index=False)

## A new approach. to be continued...

“At the end of the day, some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used.”
— Prof. Pedro Domingos

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/test.csv")

In [ ]:
cat_feat = [x for x in train.columns if train[x].name.startswith("cat")]
num_feat = [x for x in train.columns if train[x].name.startswith("cont")]

In [ ]:
def skewness(df):
    for col in num_feat:
        q3 = np.quantile(df[col], 0.75)
        q1 = np.quantile(df[col], 0.25)
        iqr = q3 - q1
        upper_limit = q3 + (1.5*iqr)
        lower_limit = q1 - (1.5*iqr)
        df = df[df[col].apply(lambda x: lower_limit <= x <= upper_limit)]
    return df

In [ ]:
train = skewness(train)

In [ ]:
train[num_feat]

### Adaptive bining

1. Quantile based bining strategy - Quantiles are a specific values or cut-points which help in partitioning continues distribution into descrete contiguous bins or intervals. q-Quantiles help us partitioning continues feature into q equal partitions. Popular examples of quantiles:

    * 2-Quantile known as median which divides the data distribution into 2 equal bins,
    * 4-Quantiles know as quartiles which divides the data into 4 equal bins,
    * 10-Quantiles asno known as deciles,

In [ ]:
#num_feat

In [ ]:
quantile_list = [0, .25, .5, .75, 1.]
quantiles = train['cont0'].quantile(quantile_list)
quantiles

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
train["cont0"].hist(bins=30, edgecolor="black", grid=False)

for quantile in quantiles:
    plt.axvline(quantile, color='red')
    
ax.set_title("Cont0 feature")

In [ ]:
quantile_labels = ["0-25Q","25-50Q","50-75Q","70-100Q"]
train["cont0_range"] = pd.qcut(train["cont0"], q=quantile_list)
train["cont0_labels"] = pd.qcut(train["cont0"], q=quantile_list, labels=quantile_labels)

In [ ]:
train[["cont0", "cont0_range", "cont0_labels"]].head()

In [ ]:
train["cont0_labels"].value_counts()

In [ ]:
sns.boxplot(x="cont0_labels", y="target", data=train)

In [ ]:
train.drop(["cont0_range", "cont0_labels"], axis=1, inplace=True)

An important point to remember here is that the resultant outcome of binning leads to discrete valued categorical features and you might need an additional step of feature engineering on the categorical data before using it in any model

In [ ]:
def numerical_binning(df):
    
    quantile_list = [0, .25, .50, .75, 1.]
    quantile_labels = ["0-25Q","25-50Q","50-75Q","75-100Q"]
    binning_df = pd.DataFrame()
    
    for col in num_feat:
        #quantiles = df[col].quantile(quantile_list)
        #binning_df[str(col)+"_range"] = pd.qcut(df[col], q=quantile_list)
        binning_df[str(col)+"_label"] = pd.qcut(df[col], q=quantile_list, labels=quantile_labels)
        
    return binning_df

In [ ]:
new_train = numerical_binning(train)
new_train.head()

In [ ]:
fig = plt.figure(figsize=(18,20))

for i, col in enumerate(new_train.columns):
    plt.subplot(8,3, i+1)
    sns.boxplot(x=new_train[col], y='target', data=train)
    plt.ylabel("target")
    plt.tight_layout()

In categorical features we have that troublesome columns "cat6", I will drop it from both datasets.

In [ ]:
train = train.drop('cat6', axis=1)
test = test.drop('cat6', axis=1)

cat_feat.remove("cat6")

So, now I have got categorical features which need to be onehotencoded and new dataframe with numerical transformed into form of categorical. Now, question is how I am suppose to treat them, as categorical nominal or categorical ordinal? I think I should transform them using methods for ordinal features.

Sklearn provides us with a method which help us categorize numerical features and also encode it as ordinal.

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

trans = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')

train[num_feat] = trans.fit_transform(train[num_feat])
test[num_feat] = trans.transform(test[num_feat])

In [ ]:
train

In [ ]:
# Get dummy variables for categorical features
train_dummies = pd.get_dummies(train[cat_feat], drop_first=True)
test_dummies = pd.get_dummies(test[cat_feat], drop_first=True)

In [ ]:
# Drop categorical columns
train.drop(cat_feat, axis=1, inplace=True)
test.drop(cat_feat, axis=1, inplace=True)

In [ ]:
# Concatenate the dataframes together
new_train = pd.concat([train_dummies, train], axis=1)
new_test = pd.concat([test_dummies, test], axis=1)

In [ ]:
id_col = new_test['id']
new_test = new_test.drop("id", axis=1)

In [ ]:
X = new_train.drop(["id", "target"], axis=1)
y = new_train['target']

In [ ]:
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=45)

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def evaluate_model(model):
    kf = KFold(n_splits=5, shuffle=True, random_state=45)
    
    n_scores = cross_val_score(model,
                               X, y,
                               scoring="neg_root_mean_squared_error",
                               verbose=2,
                               n_jobs=-1,
                               cv=kf)
    return n_scores

In [ ]:
xgboost_model = XGBRegressor()

n_scores = evaluate_model(xgboost_model)

In [ ]:
print(f"XGBoost RMSE: {np.mean(-n_scores)}")

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
k = round(np.sqrt(len(train)))

In [ ]:
knn = KNeighborsRegressor(n_neighbors=k)

knn_scores = evaluate_model(knn)

In [ ]:
print(f"KNN RMSE: {np.mean(-knn_scores)}")

**LinearSVR**

In [ ]:
lsvr_model = LinearSVR()

lsvr_scores = evaluate_model(lsvr_model)

In [ ]:
print(f"LinearSVR RMSE: {np.mean(-lsvr_scores)}")

In [ ]:
#features = [col for col in X.columns if col.startswith("c")]

In [ ]:
set(new_test.columns) == set(X.columns)

In [ ]:
X = new_train.drop(["id", "target"], axis=1)
y = new_train['target']

kf = KFold(n_splits=5, shuffle=True, random_state=45)
oof = np.zeros(len(X))
score_list = []
fold = 1
test_preds = []


for train_index, test_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    


    y_pred_list = []
    for seed in [1]:
        dtrain = lgbm.Dataset(X_train, y_train)
        dvalid = lgbm.Dataset(X_val, y_val)
        print(seed)
        params = {"objective": "regression",
                  "metric": "rmse",
                  "verbosity": -1,
                  "boosting_type": "gbdt",
                  "feature_fraction": 0.5,
                  "max_depth": 10,
                  "num_leaves": 120,
                  "lambda_l1": 2,
                  "lambda_l2": 2,
                  "learning_rate": 0.01,
                  "min_child_samples":50,
                  #"bagging_fraction": 0.7,
                  #"bagging_freq": 1, 
                  #"max_bin": 80,
                  "is_unbalance":True,
                  "subsample":0.3}
        params["seed"] = seed
        model = lgbm.train(params,
                        dtrain,
                        valid_sets=[dtrain, dvalid],
                        verbose_eval=100,
                        num_boost_round=100000,
                        early_stopping_rounds=100
                    )
        
        # Extra boosting.
        dtrain = lgbm.Dataset(X_train, y_train)
        dvalid = lgbm.Dataset(X_val, y_val)
        params = {"objective": "regression",
                  "metric": "rmse",
                  "verbosity": -1,
                  "boosting_type": "gbdt",
                  "feature_fraction": 0.5,
                  "max_depth": 10,
                  "num_leaves": 120,
                  "lambda_l1": 2,
                  "lambda_l2": 2,
                  "learning_rate": 0.003,
                  "min_child_samples":50,
                  #"bagging_fraction": 0.7,
                  #"bagging_freq": 1, 
                  #"max_bin": 80,
                  "is_unbalance":True,
                  "subsample":0.3}

        params["seed"] = seed
        model = lgbm.train(params,
                            dtrain,
                            valid_sets=[dtrain, dvalid],
                            verbose_eval=100,
                            num_boost_round=1000,
                            early_stopping_rounds=100,
                            init_model = model
                            )

    
    
        y_pred_list.append(model.predict(X_val))
        test_preds.append(model.predict(new_test))
    
   
    
    oof[test_index] = np.mean(y_pred_list,axis=0)    
    score = np.sqrt(mean_squared_error(y_val, oof[test_index]))
    score_list.append(score)
    print(f"RMSE Fold-{fold} : {score}")
    fold+=1

np.mean(score_list)

In [ ]:
print(score_list)
print(np.mean(score_list))

In [ ]:
lgbm = model.predict(new_test)

In [ ]:
first_sub = pd.DataFrame({"id":id_col,
                          "target": lgbm})

first_sub.to_csv("sub2_tbs_feb-2021.csv", index=False)

### Optuna to the rescue.

In [ ]:
import optuna

In [ ]:
def objective(trial):
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=45)
    dtrain = lgbm.Dataset(X_train, label=y_train)
    
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'dart',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int("max_depth", 1, 100),
        'max_bin': trial.suggest_int('max_bin', 1, 255)
    }
    
    gbm = lgbm.train(param, dtrain)
    preds = gbm.predict(X_val)
    pred_labels = np.rint(preds)
    rmse = np.sqrt(mean_squared_error(y_val, pred_labels))
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print("Number of finished trials: ", len(study.trials))
print("Best trial: ", study.best_trial.params)

In [ ]:
# id_col = new_test['id']

In [ ]:
X = new_train.drop(["id", "target"], axis=1)
y = new_train['target']

kf = KFold(n_splits=5, shuffle=True, random_state=45)
oof = np.zeros(len(X))
score_list = []
fold = 1
test_preds = []


for train_index, test_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    


    y_pred_list = []
    for seed in [1]:
        dtrain = lgbm.Dataset(X_train, y_train)
        dvalid = lgbm.Dataset(X_val, y_val)
        print(seed)
        params = {"objective": "regression",
                  "metric": "rmse",
                  "verbosity": -1,
                  "boosting_type": "dart", # better performance on 'gbdt'
                  "feature_fraction": 0.5577195606178571,
                  "max_depth": 27,
                  "num_leaves": 130,
                  "lambda_l1": 7.613380888310388,
                  "lambda_l2": 4.877151813351834e-06,
                  "learning_rate": 0.01,
                  "min_child_samples":45,
                  "bagging_fraction": 0.4018534965764494,
                  "bagging_freq": 5,
                  "max_bin": 149,}
                  #"is_unbalance":True,
                  #"subsample":0.3}
        params["seed"] = seed
        model = lgbm.train(params,
                        dtrain,
                        valid_sets=[dtrain, dvalid],
                        verbose_eval=100,
                        num_boost_round=2000,
                        early_stopping_rounds=100
                    )
        
        # Extra boosting.
        dtrain = lgbm.Dataset(X_train, y_train)
        dvalid = lgbm.Dataset(X_val, y_val)
        params = {"objective": "regression",
                  "metric": "rmse",
                  "verbosity": -1,
                  "boosting_type": "dart",
                  "feature_fraction": 0.5065829762803686,
                  "max_depth": 27,
                  "num_leaves": 130,
                  "lambda_l1": 5.20581102862389e-05,
                  "lambda_l2": 0.003260006773537875,
                  "learning_rate": 0.003,
                  "min_child_samples":45,
                  "bagging_fraction": 0.4018534965764494,
                  "bagging_freq": 5,
                  "max_bin": 149,}
                  #"is_unbalance":True,
                 # "subsample":0.3}

        params["seed"] = seed
        model = lgbm.train(params,
                            dtrain,
                            valid_sets=[dtrain, dvalid],
                            verbose_eval=100,
                            num_boost_round=1900,
                            early_stopping_rounds=100,
                            init_model = model
                            )

    
    
        y_pred_list.append(model.predict(X_val))
        #test_preds.append(model.predict(new_test))
    
   
    
    oof[test_index] = np.mean(y_pred_list,axis=0)    
    score = np.sqrt(mean_squared_error(y_val, oof[test_index]))
    score_list.append(score)
    print(f"RMSE Fold-{fold} : {score}")
    fold+=1

np.mean(score_list)

In [ ]:
print(score_list)
print(np.mean(score_list))

In [ ]:
preds = model.predict(new_test)

sub = pd.DataFrame({"id":id_col,
                    "target": preds})

sub.to_csv("sub2_with_optuna.csv", index=False)